In [37]:
import numpy as np

import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.
import mlflow.keras

In [38]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0
remote_server_uri = "http://127.0.0.1:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

exp_name = "reuters_keras"
mlflow.set_experiment(exp_name)

<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='reuters_keras', tags={}>

In [39]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000'

In [40]:
mlflow.keras.autolog()


In [41]:
max_words = 1000
batch_size = 32
epochs = 5

mlflow.log_params({"max_words": max_words, "batch_size": batch_size, "epochs": epochs})

In [42]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

Loading data...
8982 train sequences
2246 test sequences
46 classes


In [43]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [44]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [45]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Building model...


In [46]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/5
253/253 [==============================] - 2s 6ms/step - loss: 1.4132 - accuracy: 0.6864 - val_loss: 1.0581 - val_accuracy: 0.7697
Epoch 2/5
253/253 [==============================] - 2s 6ms/step - loss: 0.7703 - accuracy: 0.8190 - val_loss: 0.9304 - val_accuracy: 0.7964
Epoch 3/5
253/253 [==============================] - 1s 6ms/step - loss: 0.5373 - accuracy: 0.8712 - val_loss: 0.8718 - val_accuracy: 0.8009
Epoch 4/5
253/253 [==============================] - 1s 5ms/step - loss: 0.4112 - accuracy: 0.8984 - val_loss: 0.9041 - val_accuracy: 0.8120
Epoch 5/5
253/253 [==============================] - 1s 6ms/step - loss: 0.3243 - accuracy: 0.9209 - val_loss: 0.8837 - val_accuracy: 0.8142
INFO:tensorflow:Assets written to: C:\Users\SAKSHI~1.GUP\AppData\Local\Temp\tmp99pzbn9q\model\data\model\assets


In [47]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])
mlflow.log_metrics({"test_accuarcy": score[1], "test_loss": score[0]})

71/71 [==============================] - 0s 2ms/step - loss: 0.8815 - accuracy: 0.7974
Test score: 0.8814879059791565
Test accuracy: 0.7974176406860352
